# Segundo Laboratorio, iPLN

El objetivo de este laboratorio es realizar diferentes experimentos para representar y clasificar textos. Para esto se trabajará con un corpus para análisis de sentimiento, creado para la competencia [TASS 2020](http://www.sepln.org/workshops/tass/) (IberLEF - SEPLN).

## Entrega
Deberán entregar un archivo *.ipynb* con su solución, que incluya código, comentarios y respuestas a las preguntas que se incluyen al final de este notebook.

El plazo de entrega de la tarea 2 cierra el **20 de junio a las 23:59 horas**.

## Plataforma sugerida
Sugerimos que utilicen la plataforma [Google colab](https://colab.research.google.com/), que permite trabajar colaborativamente con un *notebook* de python. Al finalizar pueden descargar ese *notebook* en un archivo .ipynb, incluyendo las salidas ya ejecutadas, con la opción ```File -> Download -> Download .ipynb```.

## Aprobación del laboratorio
Para aprobar el laboratorio se exige como mínimo:
* Probar dos enfoques diferentes para la representación de tweets (uno basado en BoW y otro en word embeddings)
* Probar al menos dos modelos de aprendizaje automático con cada representación
* Comparar los resultados con los obtenidos por el modelo de pysentimiento.
El preprocesamiento, las pruebas con otras formas de representación de los tweets, los experimentos con otros modelos de aprendizaje automático, incluyendo aprendizaje profundo, entre otros posibles experimentos, no son requisito para aprobar el laboratorio, aunque aportan a la nota final.



# **Parte 1 - Carga y preprocesamiento del corpus**

`La aplicación de una etapa de preprocesamiento similar a la implementada en la tarea 1 es opcional.`

Es interesante hacer experimentos con y sin la etapa de preprocesamiento, de modo de comparar resultados (sobre el corpus de desarrollo, devel.csv) y definir si se incluye o no en la solución final.

In [ ]:
from IPython.utils.text import format_screen
# Importa las librerías necesarias
import pandas as pd
from google.colab import drive, files

drive.mount('/content/drive')

# Carga de los datasets
train_data = pd.read_csv('/content/drive/MyDrive/train.csv')
devel_data = pd.read_csv('/content/drive/MyDrive/devel.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')
stop_words_df = pd.read_csv('/content/drive/MyDrive/stop_words_esp_anasent.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Descripcion del proprocesamiento.**

La aplicación de estas técnicas facilitará el análisis posterior del corpus y mejorará la calidad de los resultados obtenidos. Las tecnicas en cuestion:

- `remove_urls`: Remplaza las URLs en un string con el string "".
- `remove_hashtags`: Remplaza los hashtags en un string con el string "".
- `remove_mentions`: Remplaza las menciones a usuarios en un string con el string "Usuario". Notar que los tweets al ser respondidos a otro usuario siempre llevan el ':' despues de la mención, consideramos esto dentro de la regex.
- `remove_numbers`: Remplaza los números en un string con el string "NUM".
- `remove_abbreviations`: Remplaza las abreviaciones comunes del español en un string con el término original, utilizando un diccionario predefinido.
- `remove_repeated_symbols`: Remueve caracteres repetidos más de dos veces en un string.
- `remove_laughts`: Remplaza diferentes expresiones de "risas" con el string "jaja".

In [ ]:
import re

# Funciones de preprocesamiento
## Remplaza urls con URL
def remove_urls(x=str): return re.sub('((https|http)?://|www\.)\S+', '', x)
## Remplaza hashtagas con HASHTAG
def remove_hashtags(x=str): return re.sub('#\w+', '', x)
## Remplaza @usuario con USER
def remove_mentions(x=str): return re.sub('@\w+(\:?)', 'usuario', x)
## Remplaza el remitente '@usuario:' con nada
def remove_writer(x=str): return re.sub('@\w+\:', '', x)
## Remplaza secuencia de digitos con NUM
def remove_numbers(x=str): return re.sub(r'\d{1,3}(?:\.\d{3})*', 'dos', x)
## Remplaza caracteres repetidos mas de dos veces
def remove_rep_symbols(x=str): return re.sub(r'(\w)\1{2,}', r'\1', x)
## Remplaza diferentes onomatopeya de 'risas'
def remove_laughts(x=str): return re.sub(r'([j|a]{1}[aeioujs]{3,}|[h|a]{1}[aeiouhs]{3,})\w*', 'jaja', x)
## Remplaza abreviaciones tipicas del espanol.
def remove_abbreviations(text: str) -> str:
  ABBREVIATIONS = {
    r"\b1ro\b": "primero", r"\b2do\b": "segundo", r"\b3ro\b": "tercero", r"\bbna\b": "buena",
    r"\bbnas\b": "buenas", r"\bbno\b": "bueno", r"\bbnos\b": "buenos", r"\bcm\b": "como", r"\bd\b": "de",
    r"\bdsd\b": "desde", r"\bdr\b(\.?)": "doctor", r"\bdnd\b": "donde", r"\bgral\b(\.?)": "general",
    r"\bhdp\b": "hijo de puta", r"\bhla\b": "hola", r"\bls\b": "los", r"\bm\b": "me", r"\bomg\b": "oh mi dios",
    r"\bp\b": "puede", r"\bpq\b": "por qué", r"\bq\b": "que", r"\bq tal\b": "qué tal", r"\bsts\b": "estás",
    r"\btb\b": "también", r"\btmb\b": "también", r"\bt\b": "te", r"\btd\b": "todo", r"\btds\b": "todos",
    r"\bwtf\b": "que carajos", r"\bx\b": "por", r"\bxfa\b": "por favor", r"\bxq\b": "porque"
  }

  for abbr, full_word in ABBREVIATIONS.items(): text = re.sub(abbr, full_word, text, flags=re.IGNORECASE)

  return text


# Arreglo de funciones definidas anteriormente que se aplican a los tweets
PIPELINE = [
  remove_urls,
  remove_mentions,
  remove_writer,
  remove_hashtags,
  remove_numbers,
  remove_abbreviations,
  remove_laughts,
  remove_rep_symbols,
  lambda x: x.lower()
]

def apply_preprocessing(tweet_list: list[str]) -> list[str]:
  clean_copy = list(tweet_list)
  for i, tweet in enumerate(clean_copy):
      for regex in PIPELINE:
          tweet = regex(tweet)

      clean_copy[i] = tweet

  return clean_copy;

train_tweets = train_data.iloc[:, 1].to_list()
P_train_tweets = apply_preprocessing(tweet_list= train_tweets)

devel_tweets = devel_data.iloc[:, 1].to_list()
P_devel_tweets = apply_preprocessing(tweet_list= devel_tweets)

test_tweets = test_data.iloc[:, 1].to_list()
P_test_tweets = apply_preprocessing(tweet_list= test_tweets)

In [ ]:
import random

# Add a random index for train_tweets
random_index_train = random.randint(0, len(train_tweets) - 1)

# Print the original and preprocessed tweet for train_tweets at the random index
print("Original train tweet:", train_tweets[random_index_train])
print("Preprocessed train tweet:", P_train_tweets[random_index_train])

# Add a random index for devel_tweets
random_index_devel = random.randint(0, len(devel_tweets) - 1)

# Print the original and preprocessed tweet for devel_tweets at the random index
print("Original devel tweet:", devel_tweets[random_index_devel])
print("Preprocessed devel tweet:", P_devel_tweets[random_index_devel])

# Add a random index for test_tweets
random_index_test = random.randint(0, len(test_tweets) - 1)

# Print the original and preprocessed tweet for test_tweets at the random index
print("Original test tweet:", test_tweets[random_index_test])
print("Preprocessed test tweet:", P_test_tweets[random_index_test])

Original train tweet: Me volvieron a dejar sola
Preprocessed train tweet: me volvieron a dejar sola
Original devel tweet: Cuenta atrás para la era Zapatero. En unos minutos, ex presidente
Preprocessed devel tweet: cuenta atrás para la era zapatero. en unos minutos, ex presidente
Original test tweet: Sólo quería hablar con gente en verda 
Preprocessed test tweet: sólo quería hablar con gente en verda 


# Parte 2 - Representación de los tweets

**Word Embeddings**

* A partir de los word embeddings, representar cada tweet como el vector promedio (mean vector) de los vectores de las palabras que lo componen.
* A partir de los word embeddings, representar cada tweet como la concatenación de los vectores de las palabras que lo componen (llevando el vector total a un largo fijo).

Se recomienda trabajar con alguna de las colecciones de word embeddings disponibles en https://github.com/dccuchile/spanish-word-embeddings. El repositorio incluye links a ejemplos y tutoriales.


Se pide que prueben al menos una opción basada en BoW y una basada en word embeddings.

In [ ]:
# Instalación de spacy para lematizar

!pip install -U spacy
!python -m spacy download es_core_news_sm

In [ ]:
# Modelo de lenguaje
import es_core_news_sm
MODEL_NAME = 'es_core_news_sm'
nlp = es_core_news_sm.load()

## **Word Embeddings**

El siguiente fragmento de código importa varias bibliotecas y carga los recursos necesarios para manipular y procesar textos en español.

Aquí está lo que hace cada línea:

1. **Importaciones**: Las primeras líneas importan varias bibliotecas necesarias para la ejecución del código.

2. **Constantes**: Luego se definen varias constantes. `MODEL_NAME` es el nombre del modelo de lenguaje español de SpaCy a cargar, `VEC_LIMIT` es el número máximo de vectores a cargar desde el archivo de vectores de palabras, `VEC_N` es la dimensión de los vectores de palabras y `VEC_NAME` es el nombre del archivo que contiene los vectores de palabras.

3. **Carga de modelos**: Finalmente, se cargan los modelos necesarios para la ejecución del código. `we` es el modelo de vectores de palabras, que se carga desde el archivo especificado por `VEC_NAME`, limitando el número de vectores a cargar a `VEC_LIMIT`. `nlp` es el modelo de lenguaje de SpaCy, que se carga especificando `MODEL_NAME`.

En resumen, este código está preparando el entorno para procesar texto en español utilizando el modelo de lenguaje de SpaCy y los vectores de palabras de Gensim. Estos vectores de palabras representarán semánticamente el texto en el espacio vectorial.

In [ ]:
# Descargar el archivo con los vectores
## La coleccion de vectores de la wikipedia tiene (#dimensions=300, #vectors=985,667) (2.4GBytes)
!wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec
# Verificar que se descargo el archivo wiki.es.vec
!ls

drive  sample_data  wiki.es.vec  wiki.es.vec.1


In [ ]:
# Intalacion de librerias necesarias.
## Word Embedding
!pip -q install numpy gensim scikit-learn
## Tokenizador
!pip install -U spacy
!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-20 23:00:08.214915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 21.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
# Importa las librerías necesarias para manipular wiki.es.vec
import logging
import pdb
import numpy as np
import spacy

from gensim.models import KeyedVectors
from numpy.linalg import norm
from sklearn.metrics import classification_report, accuracy_score

logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.INFO)

# Constantes de la realidad.
VEC_LIMIT = 500000
VEC_N = 300
VEC_NAME = 'wiki.es.vec'

# Herramientas de PLN.
## Vectores, nuestro modelo es wiki.es.vec y cargamos 100k vectores (de los +900k posibles).
we = KeyedVectors.load_word2vec_format(VEC_NAME, limit= VEC_LIMIT)

### **Representacion de los tweets como el "mean vector".**

Este código define una función llamada `tweet_mean_vector` que toma como entrada una cadena de texto (en este caso, un "tweet") y devuelve un vector que representa una sintesis del contenido semántico de ese tweet.

Para lograr esto, el código sigue los siguientes pasos:

1. **Tokenización**: Divide el tweet en palabras individuales con la ayuda de una biblioteca de procesamiento de lenguaje natural.

2. **Filtrado de tokens**: Filtra los tokens y solo retiene aquellos que están presentes en el vocabulario del modelo de embedding de palabras (representado por `we`).

3. **Extracción de vectores de palabras**: Para cada token que está en el vocabulario del modelo de word embedding, el código extrae su vector correspondiente. Estos vectores representan el significado de las palabras en un espacio de alta dimensión.

4. **Cálculo del vector medio**: Finalmente, la función calcula y devuelve la media de todos los vectores de palabras extraídos. Esta operación produce un solo vector que sirve como una representación agregada de todos los tokens en el tweet.


In [ ]:
def tweet_mean_vector(tweet):
  # Tokenizamos el tweet, es importante pasar todo a minusculas por el vocabulario del word embedding
  tokens = [x.text for x in nlp(tweet.lower())]
  # Lista de tokens que se encuentran en el vocabulario de la coleccion de vectores
  tokens_in_we_vocabulary = [token for token in tokens if token in we.key_to_index]
  if not tokens_in_we_vocabulary:
    return np.zeros(VEC_N)
  # Vectores de los tokens presentes
  tokens_vectors = [we[x] for x in tokens_in_we_vocabulary]
  # Media de todos los vectores
  return np.mean(tokens_vectors, axis=0)

### **Representacion de los tweets como "concatenacion de vectores".**

El siguiente código define una función llamada `tweet_vector_concat` que convierte un tweet en una secuencia de vectores de palabras. Esta función difiere de `tweet_vector` (discutido anteriormente) en que no calcula el vector medio de todos los vectores de palabras en un tweet. En cambio, retorna una secuencia de vectores, uno para cada token en el tweet, y limita la cantidad de vectores a un tamaño máximo predefinido (`MAX_TWEET_SIZE`).

Estos son los pasos que sigue:

1. **Tokenización**: Al igual que antes, divide el tweet en tokens y los convierte en minúsculas.

2. **Filtrado de tokens**: Selecciona solo aquellos tokens que están en el vocabulario del modelo de embedding de palabras.

3. **Extracción de vectores de palabras**: Extrae los vectores correspondientes a los tokens.

4. **Relleno/Truncamiento de la secuencia de vectores**: Para garantizar que todas las secuencias de vectores tengan la misma longitud (es decir, `MAX_TWEET_SIZE`), la función realiza dos operaciones:
   - Si un tweet tiene menos tokens que `MAX_TWEET_SIZE`, la función añade vectores nulos (vectores de ceros) hasta que la longitud de la secuencia de vectores sea igual a `MAX_TWEET_SIZE`.
   - Si un tweet tiene más tokens que `MAX_TWEET_SIZE`, la función trunca la secuencia de vectores a `MAX_TWEET_SIZE`.

5. **Concatenación**: La función finalmente concatena todos los vectores en la secuencia en un solo vector largo y lo retorna.

- **Largo del tweet**: Dado que Twitter limita los tweets a 280 caracteres y el promedio de caracteres por palabra en español es de [aproximadamente 9](https://www.um.es/lacell/aelinco/contenido/pdf/51.pdf), podríamos estimar que el número promedio de palabras por tweet estaría en el rango de 30 a 35 palabras.

En resumen, esta función transforma un tweet en un vector de longitud fija que representa la secuencia de palabras en el tweet.



In [ ]:
# Twitter tiene un limite de 280 caracteres.
# El promedio de caracteres por palabra en Espanol es 9
MAX_TWEET_SIZE = 30

def tweet_concat_vector(tweet):
  # Tokenizamos el tweet, es importante pasar todo a minusculas por el vocabulario del word embedding
  tokens = [x.text for x in nlp(tweet.lower())]
  # Lista de tokens que se encuentran en el vocabulario de la coleccion de vectores
  tokens_in_we_vocabulary = [token for token in tokens if token in we.key_to_index]
  # Vectores de los tokens presentes
  tokens_vectors = [we[x] for x in tokens_in_we_vocabulary]

  if not tokens_in_we_vocabulary:
    return np.zeros(VEC_N * MAX_TWEET_SIZE)

  # Si el tweet tiene menos del maximo lo rellenamos con vectores nulos de largo VEC_N
  while(len(tokens_vectors) < MAX_TWEET_SIZE):
    tokens_vectors.append(np.zeros(VEC_N))

  # Si el tweet tiene mas palabras que el maximo lo truncamos.
  tokens_vectors = tokens_vectors[:MAX_TWEET_SIZE]

  return np.concatenate(tokens_vectors)


### **Resultados.**


In [ ]:
# Select a random index
i = np.random.randint(0, len(test_tweets))

# Print the random tweet and its corresponding vector for the first 25 columns
print("Random Tweet:", test_tweets[i])
print("Random Tweet Mean Vector-First 25 Values:", tweet_mean_vector(test_tweets[i])[:25])
print("Random Tweet Vector- Last 25 values - 8974:8899:", tweet_concat_vector(test_tweets[i])[8974:8899])

Random Tweet: Damasiado frío Alonso (PSOE) en sus críticas al #rajoynazo1 Parece un técnico, no un político.
Random Tweet Mean Vector-First 25 Values: [-0.15085624 -0.10582194 -0.04401828 -0.04981082  0.08270106  0.09837361
  0.04047659  0.01863089  0.05943478 -0.29973823  0.09407208  0.05679687
  0.0808995  -0.01183365 -0.07609273  0.01983912  0.06629678  0.06087295
 -0.02978192 -0.00220739  0.1212978   0.0632063  -0.08818258 -0.2257691
  0.08657965]
Random Tweet Vector- Last 25 values - 8974:8899: []



# Parte 3 - Clasificación de los tweets

Para la clasificación de los tweets es posible trabajar con dos enfoques diferentes:

* Aprendizaje Automático basado en atributos: se pide probar al menos dos modelos diferentes, por ejemplo, Multi Layer Perceptron ([MLP](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)) y Support Vector Machines ([SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)), y usar al menos dos formas de representación de tweets (una basada en BoW y otra basada en word embeddings). Se publicó en eva un léxico de palabras positivas y negativas que puede ser utilizado para generar atributos.

* Aprendizaje Profundo: se recomienda experimentar con alguna red recurrente como LSTM. En este caso deben representar los tweets an base a word embeddings.

Deberán usar el corpus de desarrollo (devel.csv) para comparar resultados de diferentes experimentos, variando los valores de los hiperparámetros, la forma de representación de los tweets, el preprocesamiento, los modelos de AA, etc.

Tanto para la evaluación sobre desarrollo como para la evaluación final sobre test se usará la medida [Macro-F1](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) (promedio de la medida F1 de cada clase).

## SVM + Word Embeddings

### Librerias y Extructuras auxiliares

Este código realiza el preprocesamiento de datos para un experimento de aprendizaje automático en análisis de texto. Mapea las etiquetas de sentimientos a números, y genera dos tipos de vectores de palabras para cada tweet en los conjuntos de datos de entrenamiento y desarrollo.

In [ ]:
# Experimentos con Aprendizaje Automático y word embeddings
from sklearn import svm
import numpy as np

# Mapeamos las etiquetas a enteros
P = {
    'N': 0,
    'P': 1,
    'NONE': 2,
}

# etiquetas mapeadas para cada tweet de train y test
train_s = np.array([P[x] for x in train_data.iloc[:, 2].to_list()])
devel_s = np.array([P[x] for x in devel_data.iloc[:, 2].to_list()])

# mean y concat vectors de train y train (preprocesado)
train_tweets_vectors = [[tweet_mean_vector(x), tweet_concat_vector(x)] for x in train_tweets]
P_train_tweets_vectors = [[tweet_mean_vector(x), tweet_concat_vector(x)] for x in P_train_tweets]

# mean y concat vectors de devel y devel (preprocesado)
devel_tweets_vectors = [[tweet_mean_vector(x), tweet_concat_vector(x)] for x in devel_tweets]
P_devel_tweets_vectors = [[tweet_mean_vector(x), tweet_concat_vector(x)] for x in P_devel_tweets]

### Diferentes construcciones del modelo.

Este código entrena seis clasificadores SVM utilizando diferentes representaciones de los datos y diferentes metaparámetros:

1. Un SVM estándar con `tol=0.01` entrenado con vectores promedio de tweets sin preprocesar.
2. Un SVM estándar con `tol=0.01` entrenado con vectores promedio de tweets preprocesados.
3. Un SVM estándar con `tol=0.3` y `cache_size=4096` entrenado con vectores concatenados de tweets sin preprocesar.
4. Un SVM estándar con `tol=0.3` y `cache_size=4096` entrenado con vectores concatenados de tweets preprocesados.
5. Un SVM con `class_weight='balanced'`, `tol=0.1`, `cache_size=1024`, y `decision_function_shape='ovo'` (configuración 'G01') entrenado con vectores promedio de tweets sin preprocesar.
6. Un SVM con la misma configuración 'G01' pero `tol=0.001` y `cache_size=600` entrenado con vectores promedio de tweets preprocesados.

Los metaparámetros son:

- `tol`: Criterio de tolerancia para la detención del algoritmo. Un valor más pequeño puede llevar a una solución más precisa pero el algoritmo tardará más tiempo.
- `cache_size`: Controla el tamaño de la caché de kernel en los algoritmos SVM. Un valor mayor puede acelerar el entrenamiento en máquinas con suficiente memoria.
- `class_weight`: Maneja clases desequilibradas. Cuando se establece como 'balanced', el algoritmo ajusta los pesos de las clases de forma inversamente proporcional a las frecuencias de las clases.
- `decision_function_shape`: Determina la forma de la función de decisión en problemas de clasificación multiclase. 'ovo' significa "one-vs-one", una estrategia donde se crea un clasificador para cada par de clases.

Cada uno de estos clasificadores se entrena con el mismo corpus y conjunto de etiquetas (`train, train_s`), variando diferentes representaciones de los tweets (mean vector, concat vector). Los diferentes metaparámetros y representaciones resultan en diferentes rendimientos dependiendo del problema y los datos.

In [ ]:
# Mean Vectors: SVM -  Standar (train Sin Preprocesar)
std_svm_clf = svm.SVC(tol=0.01)
std_svm_clf.fit([vectors[0] for vectors in train_tweets_vectors], train_s)

SVC(tol=0.01)

In [ ]:
# Mean Vectors: SVM -  Standar (train Preprocesado)
std_svm_clf_P = svm.SVC(tol=0.01)
std_svm_clf_P.fit([vectors[0] for vectors in P_train_tweets_vectors], train_s)

SVC(tol=0.01)

In [ ]:
# Concat Vectors: SVM -  Standar (train Sin Preprocesar)
std_concat_svm_clf = svm.SVC(tol=0.3, cache_size=4096)
std_concat_svm_clf.fit([vectors[1] for vectors in train_tweets_vectors], train_s)

SVC(cache_size=4096, tol=0.3)

In [ ]:
# Concat Vectors: SVM -  Standar (Preprocesado)
std_concat_svm_clf_P = svm.SVC(tol=0.3, cache_size=4096)
std_concat_svm_clf_P.fit([vectors[1] for vectors in P_train_tweets_vectors], train_s)

SVC(cache_size=4096, tol=0.3)

In [ ]:
# Mean Vectors: SVM -  G01 Clasifier (train Sin Preprocesar)
G01_svm_clf = svm.SVC(class_weight='balanced', tol=0.001, cache_size=1024, decision_function_shape='ovo')
G01_svm_clf.fit([vectors[0] for vectors in train_tweets_vectors], train_s)

SVC(cache_size=1024, class_weight='balanced', decision_function_shape='ovo')

In [ ]:
# Mean Vectors: SVM -  G01 Clasifier (train Preprocesado)
G01_svm_clf_P = svm.SVC(class_weight='balanced', tol=0.001, cache_size=1024, decision_function_shape='ovo')
G01_svm_clf_P.fit([vectors[0] for vectors in P_train_tweets_vectors], train_s)

SVC(cache_size=1024, class_weight='balanced', decision_function_shape='ovo')

### Resultados


| Clasificador           | Tipo         | Macro F1 Score   |
|------------------------|--------------|------------------|
| std_svm_clf            | mean_vector  | 0.6036           |
| std_svm_clf_P          | mean_vector  | **0.6073**       |
| std_concat_svm_clf     | concat_vector| 0.5631           |
| std_concat_svm_clf_P   | concat_vector| 0.5501           |
| G01_svm_clf            | mean_vector  | 0.5920          |
| G01_svm_clf_P          | mean_vector  | 0.6047           |


In [ ]:
from sklearn.metrics import f1_score

svm_we_results = [
    {
        'clf': 'std_svm_clf',
        'type': 'mean_vector',
        'devel_prediction': std_svm_clf.predict([vectors[0] for vectors in devel_tweets_vectors]),
    },
    {
        'clf': 'std_svm_clf_P',
        'type': 'mean_vector',
        'devel_prediction': std_svm_clf_P.predict([vectors[0] for vectors in P_devel_tweets_vectors]),
    },
    {
        'clf': 'std_concat_svm_clf',
        'type': 'concat_vector',
        'devel_prediction': std_concat_svm_clf.predict([vectors[1] for vectors in devel_tweets_vectors]),
    },
    {
        'clf': 'std_concat_svm_clf_P',
        'type': 'concat_vector',
        'devel_prediction': std_concat_svm_clf_P.predict([vectors[1] for vectors in P_devel_tweets_vectors]),
    },
    {
        'clf': 'G01_svm_clf',
        'type': 'mean_vector',
        'devel_prediction': G01_svm_clf.predict([vectors[0] for vectors in devel_tweets_vectors]),
    },
    {
        'clf': 'G01_svm_clf_P',
        'type': 'mean_vector',
        'devel_prediction': G01_svm_clf_P.predict([vectors[0] for vectors in P_devel_tweets_vectors]),
    },
]


for result in svm_we_results:
    print(f"Clasificador: {result['clf']}")
    print(f"Tipo: {result['type']}")
    devel_prediction = result['devel_prediction']

    # devel_s son las etiquetas verdaderas
    macro_f1 = f1_score(devel_s, devel_prediction, average='macro')
    print(f"Macro F1 Score: {macro_f1}\n")

Clasificador: std_svm_clf
Tipo: mean_vector
Macro F1 Score: 0.6035951921293453

Clasificador: std_svm_clf_P
Tipo: mean_vector
Macro F1 Score: 0.6072846279050766

Clasificador: std_concat_svm_clf
Tipo: concat_vector
Macro F1 Score: 0.5631376073945945

Clasificador: std_concat_svm_clf_P
Tipo: concat_vector
Macro F1 Score: 0.5501043123088898

Clasificador: G01_svm_clf
Tipo: mean_vector
Macro F1 Score: 0.5974552392537711

Clasificador: G01_svm_clf_P
Tipo: mean_vector
Macro F1 Score: 0.6046903913371217



# Parte 4: Evaluación sobre test

Deben probar los mejores modelos obtenidos en la parte anterior sobre el corpus de test.

También deben comparar sus resultados con un modelo pre-entrenado para análisis de sentimientos de la biblioteca [pysentimiento](https://github.com/pysentimiento/pysentimiento) (deben aplicarlo sobre el corpus de test).



In [ ]:
# tags de test
test_s = np.array([P[x] for x in test_data.iloc[:, 2].to_list()])

# WE + SVM
## mean vectors del corpus test (preprocesado)
P_test_mean_vectors = [tweet_mean_vector(x) for x in P_test_tweets]

## Nuestros mejores modelos y sus resultados.

In [ ]:
# WE+SVM
svm_predictions = std_svm_clf_P.predict(P_test_mean_vectors)
macro_f1 = f1_score(test_s, svm_predictions, average='macro')
print(f"WE+SVM Macro F1 Score (Preproccessed Test Corpus): {macro_f1}\n")


WE+SVM Macro F1 Score (Preproccessed Test Corpus): 0.6085469536471794



## Resultados de modelo externo: pysentimiento

In [ ]:
!pip install pysentimiento
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

P = {
    'NEG': 0,
    'POS': 1,
    'NEU': 2,
}

predictions = [ P[analyzer.predict(x).output] for x in P_test_tweets ]

In [ ]:
macro_f1 = f1_score(test_s, predictions, average='macro')
print(f"Pysentimiento F1 Score (Preproccessed Test Corpus): {macro_f1}\n")

## Preguntas finales

Responda las siguientes preguntas:

### 1) ¿Qué modelos probaron para la representación de los tweets?
- Word Embeddings: mean (300) y concat vectors (9k)

### 3) ¿Qué modelos de aprendizaje automático probaron?
- svc() de sklearn



### 8) ¿Qué clase es la mejor clasificada por este enfoque? ¿Cuál es la peor? ¿Por qué piensan que sucede esto?

In [ ]:
f1 = f1_score(y_true=test_s, y_pred=svm_predictions, labels=[0,1,2], average=None)
print('NEG, POS, NEU')
print(f1)

NEG, POS, NEU
[0.6271722  0.68303914 0.51542952]


Según estos resultados:

    La clase NEG (Negativa) tiene una puntuación F1 de 0.627.
    La clase POS (Positiva) tiene una puntuación F1 de 0.683.
    La clase NEU (Neutra) tiene una puntuación F1 de 0.515.

Por lo tanto, según la puntuación F1, el modelo clasifica mejor los tweets positivos (POS), seguidos de los tweets negativos (NEG), y finalmente los tweets neutrales (NEU). Esto significa que el modelo es más preciso y tiene un mejor recall al clasificar tweets positivos en comparación con los tweets negativos y neutrales.

Una razon de la mala performance de los neutros puede ser:

Claridad del sentimiento: Los tweets positivos y negativos pueden tener sentimientos más claros y distinguibles que los neutrales. Los tweets neutrales pueden ser más sutiles y difíciles de clasificar. Esto toma otro valor cuando consideramos que los we, son sensibles a la semantica del tweet.

### 9) ¿Cómo son sus resultados en comparación con los de pysentimiento? ¿Por qué piensan que sucede esto?

`pysentimiento` arroja un macro f-score de 0.696, superior a nuestros modelos construidos en la parte anterior. Alguno de los motivos pueden ser:

- **Orientado al contexto del corpus**: El readme del repositorio lo deja bien claro pysentimiento es "*A Transformer-based library for SocialNLP tasks.*", el hecho que fuera construido para detectar el sentimiento de frases en redes sociales (SocialNLP) nos hace sospechar de un desempeño superior a nuestros modelos basados en librerias de uso general.